# Distance, driving distance and duration between two places
Various implementation in python for the Earth surface distance, travelling distance on roads and duration of such a journey. 

We will get help from the public dataset containing locations of all the world capitals from Kaggle - https://www.kaggle.com/nikitagrec/world-capitals-gps (public).

In [1]:
import pandas as pd
from geopy import distance
import requests # to call the openmap/google apis
import json
import datetime
import math
import itertools

# you may run into this problem https://stackoverflow.com/questions/61867945/python-import-error-cannot-import-name-six-from-sklearn-externals
# I have manually updated site-packages/mlrose/neural.py chaning 
# original:
# from sklearn.external import six
# to:
# import six
import mlrose # for travelling salesman problem

# that was fixed in mlrose_hiive, though the outputs are different
# import mlrose_hiive
import numpy as np

In [2]:
# load the dataframe with capitals
df = pd.read_csv("concap.csv")

# rename so that the column names are shorter and comply with PEP-8
df.rename(columns={"CountryName": "Country", "CapitalName": "capital", "CapitalLatitude": "lat", "CapitalLongitude": "lon", "CountryCode": "code", "ContinentName": "continent"}, inplace=True)
df.head(3)

,Country,capital,lat,lon,code,continent
0,Somaliland,Hargeisa,9.550000,44.050000,NaN,Africa
1,South Georgia and South Sandwich Islands,King Edward Point,-54.283333,-36.500000,GS,Antarctica
2,French Southern and Antarctic Lands,Port-aux-Français,-49.350000,70.216667,TF,Antarctica


Naming convetion of the variabled is described in PEP-8: https://www.python.org/dev/peps/pep-0008/#function-and-variable-names

There's discusion if it should be applied to the pandas columns as well, but I would suggest to do it - https://stackoverflow.com/questions/58584570/pep8-guidance-for-column-names-in-pandas-dataframe

In [3]:
# to start with let's filter only 2 capitals. Rome and Paris.
ropa = df[df["capital"].isin(["Rome","Paris"])].reset_index()
cities = ropa.copy()
cities

,index,Country,capital,lat,lon,code,continent
0,81,France,Paris,48.866667,2.333333,FR,Europe
1,110,Italy,Rome,41.900000,12.483333,IT,Europe


## Calculating the distance
The first obvious method is to use the shortest distance on the surface of Earth. You can use various approximations:

* Great-circle distance on the surface of sphere - https://en.wikipedia.org/wiki/Great-circle_distance
* Distances from geodesics since Earth is approximated as oblate ellipsoid https://en.wikipedia.org/wiki/Geodesics_on_an_ellipsoid
* Haversine formula - https://en.wikipedia.org/wiki/Haversine_formula, https://towardsdatascience.com/calculating-distance-between-two-geolocations-in-python-26ad3afe287b

You don't have to invent or even reproduce this math. The geopy.distance module already implemented all of these distnance calculation, it returns the values in kilometers (km), miles (mi), nautical miles (nm) or feet (ft).  All these methods are part of `distance` class we have already imported from geopy.
* `distance((latitude_point_1, longitude_point_1), (lat_2, lon_2))` - using geodesic on `WGS-84` ellipsoid
* `geodesic((latitude_point_1, longitude_point_1), (lat_2, lon_2))`
* `great_circle((latitude_point_1, longitude_point_1), (lat_2, lon_2))`

More info about geopy.distance https://geopy.readthedocs.io/en/stable/#module-geopy.distance

In [4]:
d = distance.distance((cities.loc[0, "lat"], cities.loc[0, "lon"]), (cities.loc[1, "lat"], cities.loc[1, "lon"]))
d, d.km, d.miles

(Distance(1107.8818760940028), 1107.8818760940028, 688.4058822066647)

In [5]:
getattr(d, "km")

1107.8818760940028

In [6]:
results = []
for f in [distance.distance, distance.great_circle, distance.geodesic]:
    for mes in ["kilometers","km","miles","mi","nautical","nm","feet","ft"]:
        d = f((cities.loc[0, "lat"], cities.loc[0, "lon"]), (cities.loc[1, "lat"], cities.loc[1, "lon"]))
        results.append({"method": f.__name__, "measurement": mes, "value": getattr(d, mes)})

# show as dataframe
results_df = pd.DataFrame(results)
results_df.pivot_table(index="method", columns="measurement", values="value")

measurement,feet,ft,kilometers,km,mi,miles,nautical,nm
method,,,,,,,,
geodesic,3.634783e+06,3.634783e+06,1107.881876,1107.881876,688.405882,688.405882,598.208356,598.208356
great_circle,3.630457e+06,3.630457e+06,1106.563205,1106.563205,687.586498,687.586498,597.496331,597.496331


`distance.distance` nativelly calls `distance.geodesic` that's why these two calues collapse into one row in the results. 

In [7]:
# the distance for various ellipsiods
for ellipsoid in distance.ELLIPSOIDS:
    for mes in ["kilometers","km","miles","mi","nautical","nm","feet","ft"]:
        d = distance.geodesic((cities.loc[0, "lat"], cities.loc[0, "lon"]), (cities.loc[1, "lat"], cities.loc[1, "lon"]), ellipsoid=ellipsoid)
        results.append({"method": f"geodesic: {ellipsoid}", "measurement": mes, "value": getattr(d, mes)})

# show as dataframe
results_df = pd.DataFrame(results)
results_df.pivot_table(index="method", columns="measurement", values="value")

measurement,feet,ft,kilometers,km,mi,miles,nautical,nm
method,,,,,,,,
geodesic,3.634783e+06,3.634783e+06,1107.881876,1107.881876,688.405882,688.405882,598.208356,598.208356
geodesic: Airy (1830),3.634455e+06,3.634455e+06,1107.781964,1107.781964,688.343800,688.343800,598.154408,598.154408
geodesic: Clarke (1880),3.634851e+06,3.634851e+06,1107.902624,1107.902624,688.418774,688.418774,598.219559,598.219559
geodesic: GRS-67,3.634796e+06,3.634796e+06,1107.885873,1107.885873,688.408366,688.408366,598.210515,598.210515
geodesic: GRS-80,3.634783e+06,3.634783e+06,1107.881876,1107.881876,688.405882,688.405882,598.208356,598.208356
geodesic: Intl 1924,3.634927e+06,3.634927e+06,1107.925804,1107.925804,688.433178,688.433178,598.232075,598.232075
geodesic: WGS-84,3.634783e+06,3.634783e+06,1107.881876,1107.881876,688.405882,688.405882,598.208356,598.208356
great_circle,3.630457e+06,3.630457e+06,1106.563205,1106.563205,687.586498,687.586498,597.496331,597.496331


# Driving distance
The cities can be quite close on the surface, though natural obstacles like sea or mountain can cause that the driving distance is much longer. 

In [8]:
hest = df[df["capital"].isin(["Helsinki","Stockholm"])].reset_index()
cities = hest.copy()
d = distance.distance((cities.loc[0, "lat"], cities.loc[0, "lon"]), (cities.loc[1, "lat"], cities.loc[1, "lon"]))
d.km

397.76330968599365

Even though the distance between Helsinky, the capita of Finland and Stockholm in Sweden less than 400km, if you decide to drive it's more than 1750km and 20 hours. Even if you take ferries you will drive almost 500km. Paris is located only 1107km from Rome, but roads connecting these cities have at least 1420km. 


That's why for many application you want to know the real travel distance, which no mathematical function can return. You need to call some map service API - e.g. google routes or osrm route service (http://project-osrm.org/docs/v5.5.1/api/#route-service). The documentation says that in order to get the driving distance we need to call this API endoint - `/route/v1/{profile}/{coordinates}?alternatives={true|false}&steps={true|false}&geometries={polyline|polyline6|geojson}&overview={full|simplified|false}&annotations={true|false}` having parameters: 

* `profile` - car, bike, foot 
* `coordinates` - lat,lon of the first point; lon, lat of the second point, e.g. 2.333333,48.866667;12.483333,41.900000
* `alternative` - whether to return only the first option or more alternatives
* `steps` - whether to return route steps - e.g. at the crossroad turn left
* `geometries` - how is the route returned, either `polyline` (def`ault), `polyline6` ,  `geojson`
* `overview` - how  to return the route - `simplified` (default), `full`,  `false`
* `annotations` - if additional metadata are provided for each point on the route

For our purpose, we can run the simplest request, having everything set to false or default. We will simply call: `http://router.project-osrm.org/route/v1/driving/cities.loc[0, "lon"],cities.loc[0, "lat"];cities.loc[1, "lon"],cities.loc[1, "lat"]?overview=false`

To get the API's response, we will use the python's requests method. 

In [9]:
cities = ropa.copy()
r = requests.get(f"""http://router.project-osrm.org/route/v1/car/{cities.loc[0, "lon"]},{cities.loc[0, "lat"]};{cities.loc[1, "lon"]},{cities.loc[1, "lat"]}?overview=false""")

It returns whatever is provided by the API in the `content` parameter.

In [10]:
r.content

b'{"code":"Ok","waypoints":[{"hint":"SJu0lP___39RAAAAYQAAAAkAAAAAAAAAUTdZQvgsIUGPZrZAAAAAAFEAAABhAAAACQAAAAAAAADcFwEA45kjAJ6l6QKVmiMAa6XpAgEAvwwTttuE","distance":14.236102,"location":[2.333155,48.866718],"name":"Rue Saint-Roch"},{"hint":"F1L0jv___38VAAAAZgAAAAAAAAAAAAAA3AxzQf1MYEIAAAAAAAAAABUAAABmAAAAAAAAAAAAAADcFwEAbXu-AFBXfwIFe74A4Fd_AgAAHw8TttuE","distance":18.175415,"location":[12.483437,41.899856],"name":"Via dell\'Umilt\xc3\xa0"}],"routes":[{"legs":[{"steps":[],"weight":54305.7,"distance":1432281.4,"summary":"","duration":54279.6}],"weight_name":"routability","weight":54305.7,"distance":1432281.4,"duration":54279.6}]}'

We can see that it returns a json object, which looks nices, when passed to the python's json library. 

In [11]:
json.loads(r.content)

{'code': 'Ok',
 'waypoints': [{'hint': 'SJu0lP___39RAAAAYQAAAAkAAAAAAAAAUTdZQvgsIUGPZrZAAAAAAFEAAABhAAAACQAAAAAAAADcFwEA45kjAJ6l6QKVmiMAa6XpAgEAvwwTttuE',
   'distance': 14.236102,
   'location': [2.333155, 48.866718],
   'name': 'Rue Saint-Roch'},
  {'hint': 'F1L0jv___38VAAAAZgAAAAAAAAAAAAAA3AxzQf1MYEIAAAAAAAAAABUAAABmAAAAAAAAAAAAAADcFwEAbXu-AFBXfwIFe74A4Fd_AgAAHw8TttuE',
   'distance': 18.175415,
   'location': [12.483437, 41.899856],
   'name': "Via dell'Umiltà"}],
 'routes': [{'legs': [{'steps': [],
     'weight': 54305.7,
     'distance': 1432281.4,
     'summary': '',
     'duration': 54279.6}],
   'weight_name': 'routability',
   'weight': 54305.7,
   'distance': 1432281.4,
   'duration': 54279.6}]}

We're mainly interested about the driving `distance` and/or driving `duration`. These parameters are included into the `route` subelement which contains a list of routes. Because we haven't asked for any alternative, this list has only one item. 

In [12]:
route_1 = json.loads(r.content)["routes"][0]
route_1["distance"], route_1["duration"]

(1432281.4, 54279.6)

If the resulted numbers look strange, beware that the distance is in meters and duration in seconds. You can easily check that the values are correct online - https://www.openstreetmap.org/directions?engine=fossgis_osrm_car&route=48.867%2C2.333%3B41.900%2C12.484 (for driving distance by car from Paris to Rome). 

Maybe you prefer more human readable format. That can be achieved by passing the received duration as a parameter to `timedelta` function of `datetime` and return the string representation. In case it's more than 1 day (I've added 100000 seconds the Paris-Rome distance for that purpose), than it's displayed.  

In [13]:
x = datetime.timedelta(seconds=route_1["duration"])

In [14]:
str(x)

'15:04:39.600000'

You can also use pandas, if you specify the type of the `duration` column to be `timedelta64[s]`.

In [15]:
dftime = pd.DataFrame({"duration":route_1["duration"]+100000}, index=[0])
dftime["duration"] = dftime["duration"].astype("timedelta64[s]")
dftime

,duration
0,1 days 18:51:19


You should use the OSRM project responsibly, because it's there to server everyone. Read the rules of the service https://github.com/Project-OSRM/osrm-backend/wiki/Api-usage-policy. 

If you are building a business application which needs hundreds or thousands of route requests, opt for a commercial product like google direction service. https://developers.google.com/maps/documentation/directions/overview. Again you use the requests library, specify correct url and get the result. Beware that you need to make an agreement with Google and these requests are paid. You have to prove yourself with valid API key and you are billed according to the service policies. Usually these services offer some free amount of searches, which can be used for your proof of concept. E.g. google offer \\$200/month worth of credit, while 1K requests costs \\$10. 

In order to avoid surprises, always restrict your API keys to the purpose you need it for and set up the billing ceiling. 

In [16]:
origin_coor = ",".join([str(cities.loc[0,"lat"]), str(cities.loc[0,"lon"])])
destination_coor = ",".join([str(cities.loc[1,"lat"]), str(cities.loc[1,"lon"])])
API_KEY = "...your_google_api_key..."
url = f"https://maps.googleapis.com/maps/api/directions/json?origin={origin_coor}&destination={destination_coor}&mode=driving&key={API_KEY}"

# alternatively you can specify the start point (origin) and the destination using the places' names
url_alt = f"https://maps.googleapis.com/maps/api/directions/json?origin=Paris&destination=Rome&mode=driving&key={API_KEY}"
print(url)

https://maps.googleapis.com/maps/api/directions/json?origin=48.86666666666667,2.333333&destination=41.9,12.483333&mode=driving&key=...your_google_api_key

In [17]:
r = requests.get(url_alt)

In this case you find the results in the the appropriate keys of the response. 

In [18]:
results = json.loads(r.content)
legs = results.get("routes").pop(0).get("legs")
legs[0].get("duration"), legs[0].get("distance")

({'text': '13 hours 44 mins', 'value': 49460},
 {'text': '1,420 km', 'value': 1420429})

In [19]:
legs.pop(0).get("duration")

{'text': '13 hours 44 mins', 'value': 49460}

# Distances between multiple cities and optimal route
The route doesn't have to be limited to two cities only. Maybe you need to visit several addreses/cities and you're trying to optimize the process. Let's try on an example of exploring all the capitals of the Central Europe. 

First we will list these capitals. Looking to Wikipedia (https://en.wikipedia.org/wiki/Central_Europe) you will find that Central Europe include 9 countries - Austria, Czech Republic, Germany, Hungary, Liechtenstein, Poland, Slovenia, Slovakia, Switzerland. I've listed their ISO2 codes below. 

In [20]:
ce_countries = ["AT","CZ","DE","HU","LI","PL","SK","SI","CH"]

In [21]:
ce_cities = df[df["code"].isin(ce_countries)].reset_index(drop=True)
ce_cities

,Country,capital,lat,lon,code,continent
0,Austria,Vienna,48.200000,16.366667,AT,Europe
1,Czech Republic,Prague,50.083333,14.466667,CZ,Europe
2,Germany,Berlin,52.516667,13.400000,DE,Europe
3,Hungary,Budapest,47.500000,19.083333,HU,Europe
4,Liechtenstein,Vaduz,47.133333,9.516667,LI,Europe
5,Poland,Warsaw,52.250000,21.000000,PL,Europe
6,Slovakia,Bratislava,48.150000,17.116667,SK,Europe
7,Slovenia,Ljubljana,46.050000,14.516667,SI,Europe
8,Switzerland,Bern,46.916667,7.466667,CH,Europe


Let's wrap the OSRM distance service into a function.

In [22]:
def get_distance(point1: dict, point2: dict) -> tuple:
    """Gets distance between two points en route using http://project-osrm.org/docs/v5.10.0/api/#nearest-service"""
    
    url = f"""http://router.project-osrm.org/route/v1/driving/{point1["lon"]},{point1["lat"]};{point2["lon"]},{point2["lat"]}?overview=false&alternatives=false"""
    r = requests.get(url)
    
    # get the distance from the returned values
    route = json.loads(r.content)["routes"][0]
    return (route["distance"], route["duration"])

In [23]:
# let's try on the first two cities
# confirm that it's correct on https://www.openstreetmap.org/directions?engine=fossgis_osrm_car&route=48.208%2C16.372%3B50.087%2C14.421
get_distance({"lat": 48.200000,"lon": 16.366667}, {"lat": 50.083333,"lon": 14.466667})

(347770, 13949.1)

Now we can run the distance calculation for all the combinations of the cities. There's 
\begin{equation*} 
{9 \choose 2} 
\end{equation*}
of combinations.

In [24]:
# https://stackoverflow.com/questions/464864/how-to-get-all-possible-combinations-of-a-list-s-elements
# https://stackoverflow.com/questions/3025162/statistics-combinations-in-python
combination = itertools.combinations(list(ce_cities["capital"]),2)
len([c for c in itertools.combinations(list(ce_cities["capital"]),2)])

36

In [25]:
list(ce_cities["capital"])

['Vienna',
 'Prague',
 'Berlin',
 'Budapest',
 'Vaduz',
 'Warsaw',
 'Bratislava',
 'Ljubljana',
 'Bern']

In [26]:
# from python 3.8 you can use math.comb to get just the number. 
math.comb(9,2)

36

Let's get the distance and duration from OSRM API for all our combinations.

In [27]:
dist_array = []
for i , r in ce_cities.iterrows():
    point1 = {"lat": r["lat"], "lon": r["lon"]}
    for j, o in ce_cities[ce_cities.index != i].iterrows():
        point2 = {"lat": o["lat"], "lon": o["lon"]}
        dist, duration = get_distance(point1, point2)
        #dist = geodesic((i_lat, i_lon), (o["CapitalLatitude"], o["CapitalLongitude"])).km
        dist_array.append((i, j, duration, dist))

In [28]:
distances_df = pd.DataFrame(dist_array,columns=["origin","destination","duration(s)","distance(m)"])
distances_df

,origin,destination,duration(s),distnace(m)
0,0,1,13949.1,347770.0
1,0,2,27424.4,695795.8
2,0,3,9661.4,246594.9
3,0,4,24166.6,647192.9
4,0,5,29367.0,689921.3
...,...,...,...,...
67,8,3,41862.1,1102405.4
68,8,4,9856.0,235662.0
69,8,5,52573.8,1503176.5
70,8,6,35718.7,936757.6


In [29]:
distances_df = distances_df.merge(ce_cities[["capital"]], left_on = "origin", right_index=True).rename(columns={"capital":"origin_name"})
distances_df = distances_df.merge(ce_cities[["capital"]], left_on = "destination", right_index=True).rename(columns={"capital":"destination_name"})
distances_df

,origin,destination,duration(s),distnace(m),origin_name,destination_name
0,0,1,13949.1,347770.0,Vienna,Prague
17,2,1,13651.1,347924.9,Berlin,Prague
25,3,1,19597.1,527313.6,Budapest,Prague
33,4,1,22708.2,619664.1,Vaduz,Prague
41,5,1,26667.0,673676.6,Warsaw,Prague
...,...,...,...,...,...,...
32,4,0,24148.1,646604.5,Vaduz,Vienna
40,5,0,29492.4,682624.3,Warsaw,Vienna
48,6,0,3533.1,80739.1,Bratislava,Vienna
56,7,0,14274.3,374135.9,Ljubljana,Vienna


# Travelling Salesperson Problem
To find the optimal route between a list of points based on a parameter (e.g. duration or distance between the places) we can use the Travelling Salesperson Problem (https://en.wikipedia.org/wiki/Travelling_salesman_problem). Travelling Sales person wants to visit all the points with the minimal effort. 

Python implementation is described in the following article. https://towardsdatascience.com/solving-travelling-salesperson-problems-with-python-5de7e883d847. You can also read the documentation of the mlrose package https://mlrose.readthedocs.io/en/stable/source/tutorial2.html.


In [30]:
# we plan to visit 9 cities
length = ce_cities.shape[0]

we plan to use the list of distances (durations in our case), that's why we initialize with `distances = dist_list` param. E.g. distance between the `place 0` and `place 1` is `13949.0` second --> `(0,1,13949.0)`

In [65]:
# turn the first three columns of the dataframe into the list of tuples
dist_list = list(distances_df[["origin","destination","duration(s)"]].sort_values(by=["origin","destination"]).to_records(index=False))
dist_list[:5] + ["..."] + dist_list[-5:]

[(0, 1, 13949.1),
 (0, 2, 27424.4),
 (0, 3, 9661.4),
 (0, 4, 24166.6),
 (0, 5, 29367.),
 '...',
 (8, 3, 41862.1),
 (8, 4, 9856.),
 (8, 5, 52573.8),
 (8, 6, 35718.7),
 (8, 7, 32803.4)]

In [32]:
# we plan to use the list of distances (durations in our case), that's why we initialize with `distances = dist_list` param.
fitness_dists = mlrose.TravellingSales(distances = dist_array)

In [33]:
problem_fit = mlrose.TSPOpt(length = length, fitness_fn = fitness_dists,
                            maximize=False)

In [34]:
mlrose.genetic_alg(problem_fit, random_state = 2)

(array([5, 2, 1, 6, 3, 7, 4, 8, 0]), 168798.30000000002)

In [35]:
# Solve problem using the genetic algorithm - suboptimal solution
best_state, best_fitness = mlrose.genetic_alg(problem_fit, random_state = 2)

In [36]:
print(f"The best state found is: {best_state}, taking {best_fitness} ({str(datetime.timedelta(seconds=best_fitness))})")

The best state found is: [5 2 1 6 3 7 4 8 0], taking 168798.30000000002 (1 day, 22:53:18.300000)


In [37]:
# better but more resource intensive solutions
best_state, best_fitness = mlrose.genetic_alg(problem_fit, mutation_prob = 0.2,  max_attempts = 500, random_state = 2)

In [38]:
print(f"The best state found is: {best_state}, taking {best_fitness} ({str(datetime.timedelta(seconds=best_fitness))})")

The best state found is: [1 8 4 7 3 6 0 5 2], taking 157397.0 (1 day, 19:43:17)


The `best_state` contains the order of the places to visit. Let's map it to our list of cities to see, the order in which we can to visit them.

In [39]:
orders = {city: order for order, city in enumerate(best_state)}
orders

{1: 0, 8: 1, 4: 2, 7: 3, 3: 4, 6: 5, 0: 6, 5: 7, 2: 8}

In [40]:
ce_cities["order"] = ce_cities.index.map(orders)
ce_cities = ce_cities.sort_values(by="order")
ce_cities

,Country,capital,lat,lon,code,continent,order
1,Czech Republic,Prague,50.083333,14.466667,CZ,Europe,0
8,Switzerland,Bern,46.916667,7.466667,CH,Europe,1
4,Liechtenstein,Vaduz,47.133333,9.516667,LI,Europe,2
7,Slovenia,Ljubljana,46.050000,14.516667,SI,Europe,3
3,Hungary,Budapest,47.500000,19.083333,HU,Europe,4
6,Slovakia,Bratislava,48.150000,17.116667,SK,Europe,5
0,Austria,Vienna,48.200000,16.366667,AT,Europe,6
5,Poland,Warsaw,52.250000,21.000000,PL,Europe,7
2,Germany,Berlin,52.516667,13.400000,DE,Europe,8


Let's confirm that the distance is really the `best_fitness`. Based on the order we will add the `next_city` column and specially handle the last city which is followed by the city with order == 0 (or the minimum order).

In [41]:
ce_cities["next_city"] = ce_cities["capital"].shift(-1)

# the last connection is between the last city and the first one
ce_cities.loc[ce_cities["order"] == max(ce_cities["order"]), "next_city"] = ce_cities.loc[ce_cities["order"] == min(ce_cities["order"]), "capital"].values[0]
ce_cities

,Country,capital,lat,lon,code,continent,order,next_city
1,Czech Republic,Prague,50.083333,14.466667,CZ,Europe,0,Bern
8,Switzerland,Bern,46.916667,7.466667,CH,Europe,1,Vaduz
4,Liechtenstein,Vaduz,47.133333,9.516667,LI,Europe,2,Ljubljana
7,Slovenia,Ljubljana,46.050000,14.516667,SI,Europe,3,Budapest
3,Hungary,Budapest,47.500000,19.083333,HU,Europe,4,Bratislava
6,Slovakia,Bratislava,48.150000,17.116667,SK,Europe,5,Vienna
0,Austria,Vienna,48.200000,16.366667,AT,Europe,6,Warsaw
5,Poland,Warsaw,52.250000,21.000000,PL,Europe,7,Berlin
2,Germany,Berlin,52.516667,13.400000,DE,Europe,8,Prague


Then we join the distances in seconds from `distance_df`

In [42]:
ordered_ce_cities = ce_cities.merge(distances_df[["origin_name","destination_name","duration(s)"]], left_on=["capital","next_city"], right_on=["origin_name","destination_name"], how="left")

In [43]:
ordered_ce_cities

,Country,capital,lat,lon,code,continent,order,next_city,origin_name,destination_name,duration(s)
0,Czech Republic,Prague,50.083333,14.466667,CZ,Europe,0,Bern,Prague,Bern,30022.3
1,Switzerland,Bern,46.916667,7.466667,CH,Europe,1,Vaduz,Bern,Vaduz,9856.0
2,Liechtenstein,Vaduz,47.133333,9.516667,LI,Europe,2,Ljubljana,Vaduz,Ljubljana,24837.6
3,Slovenia,Ljubljana,46.050000,14.516667,SI,Europe,3,Budapest,Ljubljana,Budapest,17635.7
4,Hungary,Budapest,47.500000,19.083333,HU,Europe,4,Bratislava,Budapest,Bratislava,7881.1
5,Slovakia,Bratislava,48.150000,17.116667,SK,Europe,5,Vienna,Bratislava,Vienna,3533.1
6,Austria,Vienna,48.200000,16.366667,AT,Europe,6,Warsaw,Vienna,Warsaw,29367.0
7,Poland,Warsaw,52.250000,21.000000,PL,Europe,7,Berlin,Warsaw,Berlin,20528.8
8,Germany,Berlin,52.516667,13.400000,DE,Europe,8,Prague,Berlin,Prague,13651.1


Let's check that the total of this distance is really, what Travelling Salesman Problem identified as the `best_fitness`

In [44]:
ordered_ce_cities["duration(s)"].sum()

157312.7

Since the route is a cyclical you can start at any of the capitals. To get the minimal travel time, you should end in the city which is followed by the longest duration. 

In [45]:
ordered_ce_cities["duration(s)"].max()

30022.3

In [46]:
ordered_ce_cities.style.highlight_max(color = 'lightgreen', axis = 0, subset="duration(s)")

,Country,capital,lat,lon,code,continent,order,next_city,origin_name,destination_name,duration(s)
0,Czech Republic,Prague,50.083333,14.466667,CZ,Europe,0,Bern,Prague,Bern,30022.300000
1,Switzerland,Bern,46.916667,7.466667,CH,Europe,1,Vaduz,Bern,Vaduz,9856.000000
2,Liechtenstein,Vaduz,47.133333,9.516667,LI,Europe,2,Ljubljana,Vaduz,Ljubljana,24837.600000
3,Slovenia,Ljubljana,46.050000,14.516667,SI,Europe,3,Budapest,Ljubljana,Budapest,17635.700000
4,Hungary,Budapest,47.500000,19.083333,HU,Europe,4,Bratislava,Budapest,Bratislava,7881.100000
5,Slovakia,Bratislava,48.150000,17.116667,SK,Europe,5,Vienna,Bratislava,Vienna,3533.100000
6,Austria,Vienna,48.200000,16.366667,AT,Europe,6,Warsaw,Vienna,Warsaw,29367.000000
7,Poland,Warsaw,52.250000,21.000000,PL,Europe,7,Berlin,Warsaw,Berlin,20528.800000
8,Germany,Berlin,52.516667,13.400000,DE,Europe,8,Prague,Berlin,Prague,13651.100000


As you can see, in our case, if you can start in Bern and end in Ljublana, the distance would be the smallest.

In [47]:
duration_s = ordered_ce_cities["duration(s)"].sum() - ordered_ce_cities["duration(s)"].max()
duration_s, str(datetime.timedelta(seconds=duration_s))

(127290.40000000001, '1 day, 11:21:30.400000')

# Confirm that this is really the shorted possible route

In [48]:
l = [0,1,2,3,4,5,6,7,8]

In [49]:
# There's possible (n)!/2 combinations; 1/2 because path 1-2-3 is the same as 3-2-1
math.factorial(length+2)/2

19958400.0

Let's generate all the possible paths

In [50]:
perm = [l for l in itertools.permutations(range(9), 9)]
len(perm)

362880

In [62]:
perm[:5]

[(0, 1, 2, 3, 4, 5, 6, 7, 8),
 (0, 1, 2, 3, 4, 5, 6, 8, 7),
 (0, 1, 2, 3, 4, 5, 7, 6, 8),
 (0, 1, 2, 3, 4, 5, 7, 8, 6),
 (0, 1, 2, 3, 4, 5, 8, 6, 7)]

In [52]:
distances = {(int(d[0]), int(d[1])):d[2] for d in distances_df[["origin","destination","duration(s)"]].values.tolist()}
distances[(0,1)]

13949.1

In [53]:
# pick the first path
path = perm[0]

# add th first element to conclude the circular path
path = path + (path[0],)

# iterate through the path and sum all the distances
total_path_distance = 0
for i in range(len(path)-1):
    edge = (path[i], path[i+1])
    total_path_distance += distances[edge]
    print(edge, distances[edge])
print(total_path_distance)

# list comprehension
sum([distances[(path[i],path[i+1])] for i in range(len(path)-1)])

(0, 1) 13949.1
(1, 2) 13629.7
(2, 3) 33078.6
(3, 4) 33392.4
(4, 5) 45307.4
(5, 6) 27916.9
(6, 7) 16742.8
(7, 8) 32762.8
(8, 0) 32685.3
249464.99999999997


249464.99999999997

In [54]:
mn = np.inf
min_paths = []

for p in perm:
    
    # add th first element to conclude the circular path
    p = p + (p[0],)
    
    total_path_distance = sum([distances[(p[i],p[i+1])] for i in range(len(path)-1)])
    
    if total_path_distance < mn:
        mn = total_path_distance
        min_paths = [p]
    elif total_path_distance == mn:
        min_paths.append(p)
    else:
        pass
    
print(mn, len(min_paths), min_paths)

157312.7 9 [(0, 5, 2, 1, 8, 4, 7, 3, 6, 0), (1, 8, 4, 7, 3, 6, 0, 5, 2, 1), (2, 1, 8, 4, 7, 3, 6, 0, 5, 2), (3, 6, 0, 5, 2, 1, 8, 4, 7, 3), (4, 7, 3, 6, 0, 5, 2, 1, 8, 4), (5, 2, 1, 8, 4, 7, 3, 6, 0, 5), (6, 0, 5, 2, 1, 8, 4, 7, 3, 6), (7, 3, 6, 0, 5, 2, 1, 8, 4, 7), (8, 4, 7, 3, 6, 0, 5, 2, 1, 8)]


In [55]:
def path_to_df(path, cities, distances_df):
    orders = {city: order for order, city in enumerate(path[0])}
    
    cities["order"] = cities.index.map(orders)
    cities = cities.sort_values(by="order")
    
    cities["next_city"] = cities["capital"].shift(-1)
    cities["prev_city"] = cities["capital"].shift(1)

    # the last connection is between the last city and the first one
    cities.loc[cities["order"] == max(cities["order"]), "next_city"] = cities.loc[cities["order"] == min(cities["order"]), "capital"].values[0]
    cities.loc[cities["order"] == min(cities["order"]), "prev_city"] = cities.loc[cities["order"] == max(cities["order"]), "capital"].values[0]
    
    ordered_ce_cities = cities.merge(distances_df[["origin_name","destination_name","duration(s)"]], left_on=["capital","next_city"], right_on=["origin_name","destination_name"], how="left")
    
    return ordered_ce_cities["duration(s)"].sum(), ordered_ce_cities

In [56]:
total_duration, path_df = path_to_df(min_paths, ce_cities, distances_df)
path_df.style.highlight_max(color = 'lightgreen', axis = 0, subset="duration(s)")

,Country,capital,lat,lon,code,continent,order,next_city,prev_city,origin_name,destination_name,duration(s)
0,Poland,Warsaw,52.250000,21.000000,PL,Europe,1,Berlin,Vienna,Warsaw,Berlin,20528.800000
1,Germany,Berlin,52.516667,13.400000,DE,Europe,2,Prague,Warsaw,Berlin,Prague,13651.100000
2,Czech Republic,Prague,50.083333,14.466667,CZ,Europe,3,Bern,Berlin,Prague,Bern,30022.300000
3,Switzerland,Bern,46.916667,7.466667,CH,Europe,4,Vaduz,Prague,Bern,Vaduz,9856.000000
4,Liechtenstein,Vaduz,47.133333,9.516667,LI,Europe,5,Ljubljana,Bern,Vaduz,Ljubljana,24837.600000
5,Slovenia,Ljubljana,46.050000,14.516667,SI,Europe,6,Budapest,Vaduz,Ljubljana,Budapest,17635.700000
6,Hungary,Budapest,47.500000,19.083333,HU,Europe,7,Bratislava,Ljubljana,Budapest,Bratislava,7881.100000
7,Slovakia,Bratislava,48.150000,17.116667,SK,Europe,8,Vienna,Budapest,Bratislava,Vienna,3533.100000
8,Austria,Vienna,48.200000,16.366667,AT,Europe,9,Warsaw,Bratislava,Vienna,Warsaw,29367.000000


In [57]:
total_duration

157312.7

Let's shift the order so that the path start in Ljubljana

In [58]:
filter_cities_before_bern = path_df["order"] < path_df.loc[path_df["capital"]=="Ljubljana","order"].values[0]
path_df.loc[filter_cities_before_bern, "order"] += path_df["order"].max()
path_df.sort_values(by="order")[["order", "origin_name", "destination_name", "duration(s)"]]

,order,origin_name,destination_name,duration(s)
5,6,Ljubljana,Budapest,17635.7
6,7,Budapest,Bratislava,7881.1
7,8,Bratislava,Vienna,3533.1
8,9,Vienna,Warsaw,29367.0
0,10,Warsaw,Berlin,20528.8
1,11,Berlin,Prague,13651.1
2,12,Prague,Bern,30022.3
3,13,Bern,Vaduz,9856.0
4,14,Vaduz,Ljubljana,24837.6


Let's now reverse the order, so that we won't go from Ljublana to Vaduz, but vice versa

In [59]:
path_df_reversed = path_df.sort_values(by="order").reset_index(drop=True)
path_df_reversed = path_df_reversed.reindex(index=path_df_reversed.index[::-1])
path_df_reversed.rename(columns={"destination_name":"origin_name", "origin_name":"destination_name"}, inplace=True)
path_df_reversed[["order", "origin_name", "destination_name", "duration(s)"]]

,order,origin_name,destination_name,duration(s)
8,14,Ljubljana,Vaduz,24837.6
7,13,Vaduz,Bern,9856.0
6,12,Bern,Prague,30022.3
5,11,Prague,Berlin,13651.1
4,10,Berlin,Warsaw,20528.8
3,9,Warsaw,Vienna,29367.0
2,8,Vienna,Bratislava,3533.1
1,7,Bratislava,Budapest,7881.1
0,6,Budapest,Ljubljana,17635.7


This path is exactly the same our Travelling Salesman algorithm found. You can see in the `min_paths` variable, that all combinations of these roads are the shortest (considering that we return to the place where we have started).

Draw the capitals on the Plotly Geochart. Learn more about plotly - [link](https://towardsdatascience.com/visualization-with-plotly-express-comprehensive-guide-eb5ee4b50b57)

In [60]:
import plotly.graph_objects as go

In [ ]:
# draw the capitals
fig = go.Figure(data=go.Scattergeo(
    locationmode = 'USA-states',
        lon = path_df['lon'],
        lat = path_df['lat'],
        text = df['capital'],
        mode = 'markers',
    name="capitals"
        ))

# draw the paths between the capitals
for i in range(len(path_df)-1):
    fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon=[path_df.loc[i,"lon"],path_df.loc[i+1,"lon"]], 
        lat=[path_df.loc[i,"lat"],path_df.loc[i+1,"lat"]], 
        name="-".join([path_df.loc[i,"capital"],path_df.loc[i+1,"capital"]]),
        mode="lines"))
    
# the last path
fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon=[path_df.loc[8,"lon"],path_df.loc[0,"lon"]], 
        lat=[path_df.loc[8,"lat"],path_df.loc[0,"lat"]], 
        name="-".join([path_df.loc[8,"capital"],path_df.loc[0,"capital"]]),
        mode="lines"))

fig.update_layout(
        title = 'Shortest Route Between Central European Cities',
        geo_scope='europe',
    )
fig.show()